# Process events from the Watson IoT Platform in a Streams Python application

If you have data being sent from a device to the Watson IoT platform, you can perform advanced analytics on such data using Python and the Streaming Analytics service.  This notebook shows a simple Python Streams application that connects to the Watson IoT platform to retrieve data sent there by an Edgent application.


## Table of contents
***
1.	[Prerequisites](#setup)
1.	[Create and submit the Streams application](#streams)
1.	[View results](#view)
1.  [Stop the application](#stop)
***

<a id="setup"></a>
# 1. Prerequisites


This notebook is a companion to the [Connect Apache Edgent running on Raspberry Pi to the Streaming Analytics service recipe](https://developer.ibm.com/recipes/tutorials/connect-apache-edgent-to-the-streaming-analytics-service-using-the-watson-iot-platform/).


It will show you a  Python Streams appliation that receives IoT device events from the Watson IoT platform. This is a Python version of the Java application demonstrated in that recipe. 
 

Therefore, it is assumed that you have completed steps 1-3 of the recipe. You should have the following:
- An instance of the Watson IoT platform and the Streaming Analytics service
- An Edgent application or other application that is registered with and sending events to the Watson IoT platform.
- `IotPlatformBluemix` or `IotPlatform`  application running in your Streams instance

If you are missing any of the above, please follow steps 1-3 of the recipe. Once you reach step 4, you may return to this notebook to continue.



### 1.1 Install the required version of the streamsx pacakge
This notebook requires version 1.7 or later of the streamsx package.  
Run the cell below to determine which version is installed. 

In [ ]:
!pip show streamsx

#### Update the package if needed

After running the above cell, you should see output like this:
```
---
Metadata-Version: 2.0
Name: streamsx
Version: 1.7.4
....
```
If the `Version` is not at or greater than 1.7, uncomment  and run the following cell.

In [ ]:
#Comment out the line below if needed
#!pip install --upgrade streamsx


# 2. Create and submit the Streams application

<a id="streams"></a>
### 2.1 Get your Streaming analytics service credentials
Before you create an app with this notebook, you must first provide the information that your streaming app needs to access the service. You can find this information on the Streaming Analytics service dashboard. It includes the service name, the service credentials, and the connection URL for the service. 
In the next cell you have to enter your service credentials. To copy your service credentials, open the Streaming Analytics service dashboard click **Service Credentials**, then **View Credentials**, and finally click the Copy icon.

<img src='https://github.com/orzade/streamsx-notebooks/blob/master/copyservicecredentials.png?raw=true' alt="Copy your service credentials" title="Streaming Analytics catalog - Copy your service credentials"></img>
Then paste those credentials where indicated in the cell below.

In [ ]:
import json

#Set up access to Streaming Analytics service

def get_service_name():
    ## change the service name here, it would be test1 in the screenshot above,
    # and "Streaming-Analytics" if you used the IoT starter kit
    service_name = "service_name" 
    return service_name
def get_credentials():
    credentials = """Paste Credentials here"""
 
    return credentials


"""Submit the topology to the Streaming analytics service
"""
def submit_to_service(topo):
    service_name = get_service_name()
    credentials = get_credentials()
    vs={'streaming-analytics': [{'name': service_name, 'credentials': json.loads (credentials)}]}
    cfg = {}
    cfg[ConfigParams.VCAP_SERVICES] = vs
    cfg[ConfigParams.SERVICE_NAME] = service_name
    return submit('STREAMING_ANALYTICS_SERVICE', topo, cfg)


## 2.2 Define helper functions

In [ ]:
import json
import math
from streamsx.topology.topology import Topology
from streamsx.topology.context import *
from streamsx.rest import StreamingAnalyticsConnection
from streamsx.topology import schema
import random


"""
This function will build the command tuple that is sent back
to the Edgent application. 
The commands are JSON, and the format is:

{
 'deviceId': "id of device receiving the command",
  'typeId': "typeId of receiving device",
  'cmdId': "some id",
 'jsonString': {'d':{command payload as JSON object}}
}
One way to get the device and type id is to use the data from an event we received
the incoming tuple contains the event we are responding to
"""
def get_cmd(tuple):
    #build the message you wish to send as a dictionary
    payload = {}

    payload["action"] = "Lights"
    payload["msg"] = "Message to Edgent from Streams\n:Alert code: 249"

    command_data =  {}
    command_data ["d"] = payload

    #convert the whole payload to json
    command_as_json = json.dumps(command_data)

    #build the command metadata. The device id and device type are on the tuple, but you could also specify them manually
    device_cmd ={}
    device_cmd["typeId"] = tuple["typeId"]
    device_cmd["cmdId"] = "display"
    device_cmd["deviceId"] = tuple["deviceId"]

    device_cmd["jsonString"] = command_as_json
    return device_cmd

"""Parse the data from an event we received"""
def get_event_data(tuple):
    payload_json = tuple["jsonString"]
    payload = json.loads(payload_json)
    return payload["d"]




## 2.3 Create the Streams application

### Application overview

This application is going to process events from an IoT device running an Apache Edgent application. The application has a range sensor that is determines the physical distance of an object from the sensor.  It will send events whenever an object is less than 4m away from the sensor. It sends the average, max, minimum and stddev of the distance of the object for the last 10 readings. The events are of the form:
```
{"name":"A","reading":{"N":2,"MIN":-2.384453313010427,"MAX":-2.0927794927623795,"MEAN":-2.2386164028864033,"STDDEV":0.20624453619198055}}
```
The Streams application will connect to the IoT platform to retrieve and print the events. Then it will also send a command back to the device to turn on the lights on the sensor. 

###  Create the Streams `Topology`

Streams applications are directed graphs with data moving from one operation to the next. A Streams application written in Python is called a `Topology`. The `Topology` we are creating will start by ingesting the events from the device and printing them:


In [ ]:
from streamsx.topology.topology import Topology

from streamsx.rest import StreamsConnection
from streamsx.topology import schema
import json
 #define needed variables
COMMANDS_TOPIC = "streamsx/iot/device/commands/send" #topic to publish commands to
EVENTS_TOPIC = "streamsx/iot/device/events" #topic to subscribe to for events
incoming_schema =  schema.StreamSchema("tuple <rstring typeId, rstring deviceId, rstring eventId,rstring jsonString>")
cmd_schema = schema.StreamSchema('tuple<rstring typeId, rstring deviceId, rstring cmdId, rstring jsonString>')

#Topology object is the Streams application graph
topology = Topology('ReadingsFromIot')

#Subscribe to  events
events = topology.subscribe(EVENTS_TOPIC, incoming_schema,"AllEventsAsJSON")
sensor_events = events.filter(lambda tuple: tuple["eventId"] == "sensors","SensorEventsAsJSON")
readings = sensor_events.map(get_event_data,"ReadingsStream")
#print out readings
readings.print()


#### 2.4 Send a response for each event
Next, we will create a stream of commands to send back to the device. This is a very simple scenario, since we are not performing any major analysis but simply sending a response back for each event. Typically you could do more complex analysis, or connect to a database, e.t.c. 


In [ ]:

#send a command
cmd_stream = sensor_events.map(get_cmd, "CommandsAsJSON")
#convert the commands stream to a SPL structured schema
commands_to_publish = cmd_stream.map(lambda x : (x["typeId"],x["deviceId"],x["cmdId"],x["jsonString"],), schema = cmd_schema, name="CommandsToPublish")

commands_to_publish.publish(COMMANDS_TOPIC, cmd_schema)



<a id="submit_topo"></a>
#### 2.5 Submit the `Topology` to the Streaming Analytics service

In [ ]:
result = submit_to_service(topology)
print("Submitted job to the service, job id = " + str(result.job.id))


<a id="view"></a>
# 3. View the results

#### View the events received from Edgent
Since this application prints the received events from the Edgent application to the console. You can check the log viewer in the Streams Console to see the events received from the Edgent application

#### View the commands sent back to Edgent
You can view the output of the Edgent application to see that it does receive the commands from Streams. You should see messages like this:
```
Message to Edgent from Streams:
Alert code: 249
{"name":"B","reading":{"N":50,"MIN":-2.762152936700155,"MAX":5.7895310031906675,"MEAN":2.239131357345944,"STDDEV":2.199931362960884}}
Message to Edgent from Streams:
Alert code: 249
{"name":"B","reading":{"N":50,"MIN":-2.762152936700155,"MAX":5.7895310031906675,"MEAN":2.127842287474011,"STDDEV":2.2555865482121846}}
```


<a id="stop"></a>
# 4. Stop the application
From the Streams Console, cancel this application and the `IoTPlatform` job, otherwise they might run indefinitely, consuming resources in your service. If you cancel the job you need to [re-run cell 2.5](#submit_topo).

# 5. Next Steps

Now that you have a basic Python application ingesting IoT data and running in the Streaming Analytics service, you can 

- Try the "Detect Malfunctioning Sensors" notebook:
    - [Watch the demo video on Youtube](https://www.youtube.com/watch?v=PDTSws6zULI), then 
    - [Try the notebook out yourself](https://ibm.biz/WeatherNotebook)
- Learn more about developing Streaming Analytics applications in Python: 
  - Take the [Streaming Analytics for Python developers course](https://developer.ibm.com/courses/all/streaming-analytics-basics-python-developers/) on developerWorks.
  - [Consult the Streams Python API Developer Guide](http://ibmstreams.github.io/streamsx.documentation/docs/python/1.6/python-appapi-devguide/)
- For more Edgent and IoTP documentation, you can check out the [Edgent to Quickstart guide](http://edgent.incubator.apache.org/docs/quickstart.html).
- Visit [Streamsdev, the Streams developer community](https://developer.ibm.com/streamsdev/), for more useful resources about Streams.

Happy Streaming!

 

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.